In [2]:
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import os
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from imageProcessing.ImageToArrayPreprocessor import ImageToArrayPreprocessor
from imageProcessing.SimpleDatasetLoader import SimpleDatasetLoader
from imageProcessing.aspectawarepreprocessor import AspectAwarePreprocessor
from keras.models import load_model
import pandas as pd


In [4]:
imagePaths = list()
label = list()
labels = list()

# walk through the directory to get all the images paths
for x in os.walk("ImageDownload/ImageDirectory/"):
    for filename in x[2]:
        filePath = f'{x[0]}/{filename}'
        imagePaths.append(filePath)

    s_index = x[0].rindex('/')
    # labels of classes in folder
    label.append(x[0][s_index + 1:])

# Intialize Preprocessor for image
# AspectAwarePreprocessor crop the image with respect to short dimensions
aap = AspectAwarePreprocessor(64, 64)
# ImageToArrayPreprocessor convert the image to array
iap = ImageToArrayPreprocessor()

# SimpleDatasetLoader convert image to array and process the image as per given processor
sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float") / 255.0

# convert the labels from integers to vectors
le = LabelEncoder().fit(labels)
labels = np_utils.to_categorical(le.transform(labels), len(le.classes_))
(trainX, testX, trainY, testY) = train_test_split(data,
                                                  labels, test_size=0.20, stratify=labels, random_state=42)
# load model
model = load_model("mask_googleInception_model.h5")
# load model
predictions = model.predict(testX, batch_size=64)
report = classification_report(testY.argmax(axis=1), predictions.argmax(axis=1),
                               target_names=le.classes_, output_dict=True)
# saving the classification report in csv format
df = pd.DataFrame(report).transpose()
df.to_csv("classification_report_lab.csv")

[INFO] processed 500/2370
[INFO] processed 1000/2370
[INFO] processed 1500/2370
[INFO] processed 2000/2370


In [5]:
pd.read_csv("classification_report_lab.csv")

,Unnamed: 0,precision,recall,f1-score,support
0,with_mask,0.989247,0.992806,0.991023,278.000000
1,without_mask,0.989744,0.984694,0.987212,196.000000
2,accuracy,0.989451,0.989451,0.989451,0.989451
3,macro avg,0.989495,0.988750,0.989118,474.000000
4,weighted avg,0.989453,0.989451,0.989447,474.000000
